In [ ]:
### 
### IMPORT THIS NOTEBOOK INTO A SNOWFLAKE NOTEBOOK AND RUN ALL CELLS
###

# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
create stage if not exists awesome_tools;

In [ ]:
-- optional - can replace with your region - this gives access to claude 4 and better performing models
-- ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US';

In [ ]:
# claude 4 requires cross region inference
# potentially may want to use gpt-4.1 depending on your region / cloud
model_for_generation = 'claude-4-sonnet'

In [ ]:
"""
Snowpark Python Stored Procedure to generate Streamlit apps using Snowflake Cortex Complete API
"""

import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col, sproc
import json
import os
import sys
import tempfile
import uuid


def strip_markdown_code_markers(code: str) -> str:
    """
    Strip markdown code markers from the beginning and end of the code if they exist.

    Args:
        code: The code string that might contain markdown markers

    Returns:
        str: Code with markdown markers removed
    """
    # Strip whitespace
    code = code.strip()

    # Check for markdown code block markers
    if code.startswith("```python") or code.startswith("```"):
        # Find the first newline after the opening markers
        first_newline = code.find("\n")
        if first_newline != -1:
            code = code[first_newline + 1 :]

    # Remove closing markdown markers
    if code.endswith("```"):
        # Find the last occurrence of ```
        last_markers = code.rfind("```")
        if last_markers != -1:
            code = code[:last_markers]

    # Strip any remaining whitespace
    return code.strip()


def upload_code_to_stage(
    session: snowpark.Session,
    stage_name: str,
    code: str,
    filename: str = "streamlit_app.py",
) -> bool:
    """
    Helper function to upload generated code to a Snowflake stage

    Args:
        session: Snowpark session
        stage_name: Name of the stage to upload to
        code: The Python code to upload
        filename: Name of the file to create

    Returns:
        bool: True if upload was successful, False otherwise
    """
    temp_files = []
    try:
        # Create a temporary file with the generated code
        with tempfile.NamedTemporaryFile(
            mode="w", suffix=".py", delete=False
        ) as temp_file:
            temp_file.write(code)
            temp_file_path = temp_file.name

        # Rename the temp file to the desired filename before uploading
        target_path = os.path.join(os.path.dirname(temp_file_path), filename)
        os.rename(temp_file_path, target_path)
        temp_files.append(target_path)

        # Create environment.yml file
        environment_yml_content = """name: sf_env
channels:
- snowflake
dependencies:
- plotly
- altair
"""

        with tempfile.NamedTemporaryFile(
            mode="w", suffix=".yml", delete=False
        ) as env_temp_file:
            env_temp_file.write(environment_yml_content)
            env_temp_file_path = env_temp_file.name

        # Rename the environment temp file
        env_target_path = os.path.join(
            os.path.dirname(env_temp_file_path), "environment.yml"
        )
        os.rename(env_temp_file_path, env_target_path)
        temp_files.append(env_target_path)

        # Upload the main Python file
        put_result = session.file.put(
            local_file_name=target_path,
            stage_location=f"@{stage_name}",
            auto_compress=False,
            overwrite=True,
        )

        # Upload the environment.yml file
        env_put_result = session.file.put(
            local_file_name=env_target_path,
            stage_location=f"@{stage_name}",
            auto_compress=False,
            overwrite=True,
        )

        # Clean up temporary files
        for temp_file in temp_files:
            os.unlink(temp_file)

        # Check if both uploads were successful
        main_upload_success = len(put_result) > 0 and put_result[0].status == "UPLOADED"
        env_upload_success = (
            len(env_put_result) > 0 and env_put_result[0].status == "UPLOADED"
        )

        return main_upload_success and env_upload_success

    except Exception as e:
        # Clean up temporary files if they exist
        for temp_file in temp_files:
            try:
                os.unlink(temp_file)
            except:
                pass
        return False


@sproc(name="generate_streamlit",
       is_permanent=True,
       stage_location="@awesome_tools",
       replace=True,
       packages = ["snowflake-snowpark-python", "snowflake-ml-python"])
def generate_streamlit_app(session: snowpark.Session, content: str) -> str:
    """
    Snowpark stored procedure that calls Snowflake Cortex Complete API to generate a Streamlit app
    based on the provided content.

    Args:
        session: Snowpark session
        content: Long string containing the content/requirements for the Streamlit app

    Returns:
        JSON string containing the generated Streamlit app code and metadata
    """
    try:
        # Get current Snowflake context information
        current_database = session.sql("SELECT CURRENT_DATABASE()").collect()[0][0]
        current_schema = session.sql("SELECT CURRENT_SCHEMA()").collect()[0][0]
        current_warehouse = session.sql("SELECT CURRENT_WAREHOUSE()").collect()[0][0]
        current_account = session.sql("SELECT CURRENT_ACCOUNT_NAME()").collect()[0][0]
        current_org = session.sql("SELECT CURRENT_ORGANIZATION_NAME()").collect()[0][0]
        
        # Set variables for use in the procedure
        database = current_database
        schema = current_schema
        warehouse = current_warehouse
        account_name = current_account
        org_name = current_org
        
        # Import Snowflake Cortex Complete
        from snowflake.cortex import Complete, CompleteOptions

        # Create the prompt for generating Streamlit app
        system_prompt = """You are an expert Python developer specializing in Streamlit applications for data visualization and analysis. 
        Generate a complete, functional Streamlit application based on the provided content. The app should:
        1. Be self-contained and ready to run
        2. Include proper imports
        3. Have clear structure and comments
        4. Include data visualization where appropriate
        5. Be user-friendly with proper titles and descriptions
        6. Handle errors gracefully
        
        Return only the Python code for the Streamlit app, no additional text or markdown formatting."""

        user_prompt = f"""Based on the following content, create a Streamlit application:

{content}

Requirements:
- Create an interactive dashboard if the content contains data
- Use appropriate Streamlit components (charts, tables, filters, etc.)
- Include a clear title and description
- Make it visually appealing and user-friendly
- Add any necessary data processing or analysis
"""

        # Combine system and user prompts for Cortex Complete
        full_prompt = f"{system_prompt}\n\nUser Request: {user_prompt}"

        # Call Snowflake Cortex Complete API
        # Available models: 'snowflake-arctic', 'llama3-8b', 'claude-4-sonnet', 'reka-flash', 'mistral-large', 'mixtral-8x7b', 'llama2-70b-chat', 'gemma-7b'
        response = Complete(
            model=model_for_generation,  # Using claude-4-sonnet for high-quality code generation
            prompt=full_prompt,
            session=session,
            options=CompleteOptions(max_tokens=8192),
        )

        # Extract the generated code
        streamlit_code = response

        # Strip markdown code markers if they exist
        streamlit_code = strip_markdown_code_markers(streamlit_code)

        # Generate a unique app name with timestamp
        timestamp = session.sql("SELECT CURRENT_TIMESTAMP()").collect()[0][0]
        app_name = f"SI_APP_{timestamp.strftime('%Y%m%d_%H%M%S')}"
        stage_name = f"SI_STREAMLIT_STAGE"

        # Create Streamlit app in Snowflake
        streamlit_app_url = None
        streamlit_creation_success = False
        streamlit_error = None

        try:

            # Create a stage for the Streamlit files
            create_stage_sql = f"CREATE STAGE IF NOT EXISTS {stage_name}"
            session.sql(create_stage_sql).collect()

            # Upload the generated code to the stage
            upload_success = upload_code_to_stage(
                session, stage_name, streamlit_code, filename=f"{app_name}.py"
            )

            if upload_success:
                # Create the Streamlit app using SQL
                create_streamlit_sql = f"""
                CREATE STREAMLIT {app_name}
                FROM @{stage_name}
                MAIN_FILE = '{app_name}.py'
                QUERY_WAREHOUSE = {warehouse}
                """

                # Execute the CREATE STREAMLIT command
                session.sql(create_streamlit_sql).collect()

                # Construct the Streamlit app URL
                streamlit_app_url = f"https://app.snowflake.com/{org_name}/{account_name}/#/streamlit-apps/{database}.{schema}.{app_name}"
                streamlit_creation_success = True
            else:
                streamlit_error = "Failed to upload code to stage"
                streamlit_creation_success = False

        except Exception as streamlit_error_obj:
            streamlit_error = str(streamlit_error_obj)
            streamlit_creation_success = False

        # Create response object
        result = {
            "success": True,
            "error": None,
            "streamlit_code": streamlit_code,
            "streamlit_app": {
                "created": streamlit_creation_success,
                "app_name": app_name if streamlit_creation_success else None,
                "stage_name": stage_name if streamlit_creation_success else None,
                "database": database,
                "schema": schema,
                "url": streamlit_app_url,
                "error": streamlit_error,
                "instructions": "If the app was created successfully, you can access it via the URL above or through the Snowflake web interface under Streamlit Apps.",
            },
            "metadata": {
                "model_used": "claude-4-sonnet",
                "content_length": len(content),
                "generated_at": timestamp.isoformat(),
                "prompt_length": len(full_prompt),
            },
        }

        return json.dumps(result, indent=2)

    except Exception as e:
        # Return error information
        error_result = {
            "success": False,
            "error": str(e),
            "error_type": type(e).__name__,
            "streamlit_code": None,
        }
        return json.dumps(error_result, indent=2)


print('Stored Procedure "generate_streamlit(varchar) created."')


In [ ]:
result_df = session.sql(
    "call generate_streamlit('a simple streamlit app that says hello world')"
).collect()

# Extract result from the returned Row
# Stored procs in Snowflake return a single column named 'GENERATE_STREAMLIT(...)'
result_str = result_df[0][0]

# Display it as JSON in Streamlit
try:
    json_obj = json.loads(result_str)
except json.JSONDecodeError:
    json_obj = {"result": result_str}

# Try to extract and show the URL
streamlit_url = (
    json_obj.get('streamlit_app', {}).get('url')
    if isinstance(json_obj, dict)
    else None
)

if streamlit_url:
    st.write('Streamlit app URL: ', streamlit_url)
else:
    st.warning("No Streamlit app URL found in result.")

# Show full result
st.write("Raw JSON Output:", json_obj)